# Simulación Monte Carlo para Blackjack

## Banco de Guatemala
## Programa de Estudios Superiores -PES-, 2021-2022

*Programación I*

*Profesor: Rodrigo Chang*

*Fecha: 21 de Enero de 2022*

    Luis González

    Javier Monroy

### Paquetes necesarios

In [2]:
import numpy as np
import pandas as pd

### Funciones auxiliares

In [3]:
def arrays_to_df(pfh:np.ndarray, res:np.ndarray, dfh:np.ndarray):
    s_pfh=pd.Series(pfh)
    s_res=pd.Series(res)
    s_dfh=pd.Series(dfh)
    basic_strategy_stand = pd.DataFrame({'Hard_Total':s_pfh, 'Result':s_res, 'Dealer_Upcard':s_dfh})
    return basic_strategy_stand

### El juego de Blackjack

In [11]:
def blackjack_game(decks:int, players=1):    
    profit_dealer=0
    profit_player=0
    shoe=np.full(decks*4,11)
    two=np.full(decks*4,2)
    three=np.full(decks*4,3)
    four=np.full(decks*4,4)
    five=np.full(decks*4,5)
    six=np.full(decks*4,6)
    seven=np.full(decks*4,7)
    eight=np.full(decks*4,8)
    nine=np.full(decks*4,9)
    ten=np.full(decks*4,10)
    j=np.full(decks*4,10)
    q=np.full(decks*4,10)
    k=np.full(decks*4,10)
    total_faces=(two, three, four, five, six, seven, eight, nine, ten, j, q, k)
    for i in total_faces:
        shoe = np.hstack((shoe,i))
    np.random.shuffle(shoe)
    player_hand=np.full(1,0)
    dealer_hand=np.full(1,0)
    player_first_cards = np.full(1,0)
    dealer_first_cards = np.full(1,0)
    hand_result=np.full(1,0)
    while (shoe.size>60):        
        card = int(shoe[0])
        if (player_hand.size<=2):
            player_hand=np.append(player_hand, shoe[0])
        shoe = np.delete(shoe,0)                
        card = int(shoe[0])        
        if (dealer_hand.size<=2):
            dealer_hand=np.append(dealer_hand, shoe[0])        
        shoe = np.delete(shoe,0)         
        if (player_hand.size>=3) and (dealer_hand.size>=3):            
            while (np.sum(dealer_hand)<17):
                card = int(shoe[0])                
                dealer_hand=np.append(dealer_hand, shoe[0])  
                shoe = np.delete(shoe,0)                                        
            if (np.sum(dealer_hand)==21) and (np.sum(player_hand)!=21):
                profit_dealer+=1
                profit_player-=1
                player_first_cards=np.append(player_first_cards,np.sum(player_hand))
                dealer_first_cards=np.append(dealer_first_cards, dealer_hand[1])                
                hand_result=np.append(hand_result, -1)                
                player_hand=np.full(1,0)
                dealer_hand=np.full(1,0)                                
            elif (np.sum(dealer_hand) > np.sum(player_hand)) and (np.sum(dealer_hand)<21):                
                profit_dealer+=1
                profit_player-=1
                player_first_cards=np.append(player_first_cards,np.sum(player_hand))
                dealer_first_cards=np.append(dealer_first_cards, dealer_hand[1])
                hand_result=np.append(hand_result, -1)
                player_hand=np.full(1,0)
                dealer_hand=np.full(1,0)                                
            elif (np.sum(dealer_hand) == np.sum(player_hand)): 
                player_first_cards=np.append(player_first_cards,np.sum(player_hand))
                dealer_first_cards=np.append(dealer_first_cards, dealer_hand[1])
                hand_result=np.append(hand_result, 0)
                player_hand=np.full(1,0)
                dealer_hand=np.full(1,0)                                
            elif (np.sum(player_hand)==21) and (np.sum(dealer_hand)!=21):                
                profit_dealer-=1.5
                profit_player+=1.5
                player_first_cards=np.append(player_first_cards,np.sum(player_hand))
                dealer_first_cards=np.append(dealer_first_cards, dealer_hand[1])
                hand_result=np.append(hand_result, 1.5)
                player_hand=np.full(1,0)
                dealer_hand=np.full(1,0)                                
            elif (np.sum(dealer_hand) < np.sum(player_hand)) and (np.sum(player_hand)<21):                
                profit_dealer-=1
                profit_player+=1
                player_first_cards=np.append(player_first_cards,np.sum(player_hand))
                dealer_first_cards=np.append(dealer_first_cards, dealer_hand[1])
                hand_result=np.append(hand_result, 1)
                player_hand=np.full(1,0)
                dealer_hand=np.full(1,0)                                
            elif (np.sum(dealer_hand) > 21) and (np.sum(player_hand)<21):                
                profit_dealer-=1
                profit_player+=1
                player_first_cards=np.append(player_first_cards,np.sum(player_hand))
                dealer_first_cards=np.append(dealer_first_cards, dealer_hand[1])
                hand_result=np.append(hand_result, 1)
                player_hand=np.full(1,0)
                dealer_hand=np.full(1,0)                              
    else:
        pass         
    return (arrays_to_df(player_first_cards, hand_result, dealer_first_cards))

### El experimento Monte Carlo

In [12]:
def monte_carlo_blackjack(n:int, N:int):
    Basic_Strategy = pd.DataFrame()
    for _ in range(N):
        BJiteration = blackjack_game(n)
        Basic_Strategy=pd.concat([Basic_Strategy,BJiteration], ignore_index=True)
    Basic_Strategy = Basic_Strategy.iloc[1:,:]
    return Basic_Strategy

In [13]:
BJT = monte_carlo_blackjack(6,500)
BJT

,Hard_Total,Result,Dealer_Upcard
1,19,-1.0,10
2,15,-1.0,2
3,15,-1.0,2
4,13,1.0,10
5,18,1.0,7
...,...,...,...
23487,12,-1.0,10
23488,14,-1.0,10
23489,6,1.0,8
23490,19,-1.0,10


### Nuestra Basic Strategy

In [15]:
Gamblers_Strategy = BJT.pivot_table(index="Hard_Total",columns="Dealer_Upcard").iloc[1:18,1:]
Gamblers_Strategy

Result                                                    \
Dealer_Upcard        2         3         4         5         6         7    
Hard_Total                                                                  
4             -0.200000  0.142857  0.333333  0.111111 -0.111111 -0.833333   
5             -0.684211 -0.333333 -0.363636  0.052632  0.444444 -0.575758   
6             -0.176471 -0.411765 -0.023256 -0.200000  0.000000 -0.230769   
7             -0.176471 -0.234043  0.019608  0.315789  0.061224 -0.245283   
8             -0.083333 -0.045455 -0.090909 -0.034483 -0.137931 -0.183673   
9             -0.107692 -0.115385  0.034483  0.137931  0.031250 -0.555556   
10            -0.168539 -0.353846 -0.076923 -0.114286  0.000000 -0.289474   
11             0.050000 -0.108911 -0.259259 -0.055556 -0.076923 -0.342466   
12            -0.226667  0.024096 -0.093923 -0.014493 -0.006711 -0.430464   
13            -0.218391 -0.165714 -0.093023 -0.112360 -0.116883 -0.314607   
14            -0.125000 -0.149425 -0.223684 -0.065089 -0.005848 -0.283582   
15            -0.295597 -0.083871  0.014286  0.006369 -0.194444 -0.502959   
16            -0.138211 -0.114754  0.000000 -0.218045 -0.157143 -0.459016   
17            -0.058824  0.092308  0.032258 -0.077519  0.057143 -0.163934   
18             0.066038  0.311321  0.300000  0.364341  0.228814  0.401639   
19             0.442308  0.514286  0.394737  0.500000  0.556522  0.701031   
20             0.693467  0.736842  0.726316  0.637681  0.704301  0.775000   

                                                       
Dealer_Upcard        8         9         10        11  
Hard_Total                                             
4             -1.000000 -0.714286 -0.680000 -0.166667  
5             -0.444444 -0.280000 -0.531915 -0.375000  
6             -0.448276 -0.928571 -0.471074 -0.478261  
7             -0.365854 -0.589744 -0.416667 -0.478261  
8             -0.419355 -0.510204 -0.519608 -0.355932  
9             -0.448276 -0.473684 -0.539095 -0.386667  
10            -0.310345 -0.616438 -0.594855 -0.679012  
11            -0.360000 -0.581395 -0.652174 -0.569892  
12            -0.418440 -0.376471 -0.510563 -0.443038  
13            -0.452514 -0.484277 -0.552100 -0.631285  
14            -0.563910 -0.461538 -0.520611 -0.396450  
15            -0.373333 -0.379310 -0.506667 -0.417219  
16            -0.605263 -0.500000 -0.519713 -0.586667  
17            -0.400000 -0.403226 -0.464503 -0.433071  
18             0.144068 -0.298387 -0.226981 -0.234694  
19             0.706897  0.247863 -0.037037  0.137615  
20             0.802198  0.831633  0.398374  0.153846